In [0]:
!pip install geopy

Collecting geopy
 Downloading geopy-2.3.0-py3-none-any.whl (119 kB)
██▊ | 10 kB 26.9 MB/s eta 0:00:01 |█████▌ | 20 kB 27.3 MB/s eta 0:00:01 |████████▏ | 30 kB 33.8 MB/s eta 0:00:01 |███████████ | 40 kB 14.3 MB/s eta 0:00:01 |█████████████▊ | 51 kB 13.4 MB/s eta 0:00:01 |████████████████▍ | 61 kB 14.5 MB/s eta 0:00:01 |███████████████████▏ | 71 kB 15.6 MB/s eta 0:00:01 |█████████████████████▉ | 81 kB 17.1 MB/s eta 0:00:01 |████████████████████████▋ | 92 kB 18.3 MB/s eta 0:00:01 |███████████████████████████▍ | 102 kB 17.6 MB/s eta 0:00:01 |██████████████████████████████ | 112 kB 17.6 MB/s eta 0:00:01 |████████████████████████████████| 119 kB 17.6 MB/s 
Collecting geographiclib<3,>=1.52
 Downloading geographiclib-2.0-py3-none-any.whl (40 kB)
████████▏ | 10 kB 39.2 MB/s eta 0:00:01 |████████████████▎ | 20 kB 46.6 MB/s eta 0:00:01 |████████████████████████▍ | 30 kB 55.6 MB/s eta 0:00:01 |████████████████████████████████| 40 kB 7.7 MB/s 
Installing collected packages: geographiclib, geopy
Successfully installed geographiclib-2.0 geopy-2.3.0
 WARNING: You are using pip version 21.0.1; however, version 22.3.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
import pyspark.sql.functions as F
from functools import reduce

# METEOCAT

## Union de las  Variables y Valores Climatologicos de METEOCAT

- **Pivot de variablesValoresClimaDF**

    Para que cada fila de la variable sea una columna.

- **Se renombran las columnas de la tabla pivoteada de METEOCAT para que sean igual que el DF de AEMET**
- **Se añaden el nombre de estación, municipio y la provincia a la tabla Pivoteada.**

In [0]:
variablesValoresClimaDF = spark.read.format("parquet") \
.load("/mnt/IncendiosForestalesCAT/raw/meteocat/variables")


valoresClimaMeteocatDf = spark.read.format("parquet") \
.load("/mnt/IncendiosForestalesCAT/raw/meteocat/valores_clima_diarios")

variablesValoresClimaDF = variablesValoresClimaDF.alias("vmeteo")\
.join(valoresClimaMeteocatDf.alias("vcmd_meteo"), F.col("vmeteo.codi_variable") == F.col("vcmd_meteo.codi_variable"), "inner")\
.select(["vmeteo.nom_variable", "vcmd_meteo.valor_lectura", "vmeteo.unitat", "vcmd_meteo.codi_estacio", "vcmd_meteo.data_lectura"])

# Pivot
variables = ['Pressió atmosfèrica mínima', 
             'Pressió atmosfèrica', 
             'Pressió atmosfèrica màxima', 
             'Direcció de vent 10 m (m. 1)', 
             'Direcció del vent a 6 m (m. 1)',
             'Direcció del vent a 2 m (m. 1)', 
             'Velocitat del vent a 10 m (esc.)', 
             'Velocitat del vent a 2 m (esc.)', 
             'Velocitat del vent a 6 m (esc.)', 
             'Ratxa màxima del vent a 10 m', 
             'Ratxa màxima del vent a 2 m', 
             'Ratxa màxima del vent a 6 m', 
             'Direcció de la ratxa màxima del vent a 10 m',
             'Direcció de la ratxa màxima del vent a 2 m', 
             'Direcció de la ratxa màxima del vent a 6 m', 
             'Precipitació', 
             'Temperatura', 
             'Temperatura mínima', 
             'Humitat relativa',  
             'Humitat relativa mínima', 
             'Humitat relativa màxima',
             'Precipitació màxima en 1 minut',
             'Irradiància solar global',
             'Irradiància neta',
             'Temperatura màxima',
             'Gruix de neu a terra']

variablesValoresClimaDF = variablesValoresClimaDF.withColumn("valor_lectura", F.col("valor_lectura").cast("float"))

pivotDF = variablesValoresClimaDF.groupBy("data_lectura", "codi_estacio").pivot("nom_variable", variables).sum("valor_lectura")\
.withColumn("velmedia", F.concat_ws("",F.col("`Velocitat del vent a 10 m (esc.)`"), 
                                                     F.col("`Velocitat del vent a 2 m (esc.)`"),
                                                     F.col("`Velocitat del vent a 6 m (esc.)`")).cast("float")) \
                  .withColumn("racha", F.concat_ws("",F.col("`Ratxa màxima del vent a 10 m`"), 
                                 F.col("`Ratxa màxima del vent a 6 m`"),
                                 F.col("`Ratxa màxima del vent a 2 m`")).cast("float"))

rename_columns = {"fecha":"data_lectura",
"prec":"Precipitació",
"presMax": "Pressió atmosfèrica màxima",
"presMin": "Pressió atmosfèrica mínima",
"sol" :"Irradiància solar global",
"tmax": "Temperatura màxima",
"tmed": "Temperatura",
"tmin": "Temperatura mínima",
"indicativo":"codi_estacio",
"rhum":"Humitat relativa",
"velmedia":"velmedia",
"racha":"racha"}

pivotRenamedDF = reduce(lambda dfl, x: dfl.withColumnRenamed(rename_columns[x], x),rename_columns, pivotDF).select(*rename_columns.keys())

columns = ["fecha", "indicativo", "nombre","provincia", "prec", "tmax", "tmed", "tmin", "velmedia", "racha", "sol", "presMax", "presMin", "rhum", "municipio", "cod_municipio"]

estaciones_meteocatDF = spark.read.parquet("/mnt/IncendiosForestalesCAT/raw/meteocat/estaciones/")
pivotRenamedDF = pivotRenamedDF.alias("valoresMeteo").join(estaciones_meteocatDF.alias("eMeteo"), pivotRenamedDF.indicativo ==estaciones_meteocatDF.codi_estacio, "inner" )\
.select("valoresMeteo.*","eMeteo.nom_provincia", "eMeteo.nom_estacio","eMeteo.codi_municipi", "eMeteo.nom_municipi", "eMeteo.codi_provincia") \
.withColumnRenamed("nom_provincia","provincia") \
.withColumnRenamed("nom_estacio","nombre") \
.withColumnRenamed("nom_municipi","municipio") \
.withColumn("cod_municipio",F.concat(F.col("codi_provincia"),F.col("codi_municipi"))) \
.select(columns)

display(pivotRenamedDF)


fecha,indicativo,nombre,provincia,prec,tmax,tmed,tmin,velmedia,racha,sol,presMax,presMin,rhum,municipio,cod_municipio
2016-10-25T12:00:00.000,C9,Mas de Barberans,Tarragona,0.0,21.200000762939453,20.700000762939453,20.299999237060547,1.8,3.1,414.0,992.5,992.2000122070312,86.0,Mas de Barberans,43519
2016-11-22T12:00:00.000,VS,Lac Redon (2.247 m),Lleida,0.0,-0.30000001192092896,-0.30000001192092896,-0.30000001192092896,7.0,16.1,85.0,766.7999877929688,765.7999877929688,100.0,Vielha e Mijaran,25879
2016-11-23T12:00:00.000,C8,Cervera,Lleida,0.0,15.600000381469727,15.399999618530273,15.199999809265137,8.4,14.1,120.0,939.0999755859375,938.0,75.0,Cervera,25720
2016-11-15T12:00:00.000,KP,Fogars de la Selva,Girona,0.0,16.399999618530273,16.0,15.5,null,null,null,null,null,46.0,Fogars de la Selva,17786
2016-11-26T12:00:00.000,W4,la Granada,Barcelona,0.0,11.0,10.899999618530273,10.699999809265137,3.3,5.1,158.0,null,null,76.0,La Granada,898
2016-12-04T12:00:00.000,W8,Blancafort,Tarragona,0.0,13.100000381469727,12.600000381469727,12.199999809265137,null,null,null,null,null,81.0,Blancafort,43385
2016-12-27T12:00:00.000,X9,Caldes de Montbui,Barcelona,0.0,14.699999809265137,14.100000381469727,13.699999809265137,1.8,2.9,432.0,1011.2000122070312,1010.9000244140625,58.0,Caldes de Montbui,8907
2017-01-19T12:00:00.000,UF,PN del Garraf - el Rascler,Barcelona,0.0,6.5,5.599999904632568,5.0,4.8,8.8,228.0,null,null,60.0,Begues,8288
2017-02-06T12:00:00.000,X9,Caldes de Montbui,Barcelona,0.0,14.800000190734863,14.399999618530273,14.0,5.5,8.8,601.0,994.0999755859375,993.5999755859375,30.0,Caldes de Montbui,8907
2017-02-16T12:00:00.000,WR,Torroja del Priorat,Tarragona,0.0,15.399999618530273,15.100000381469727,14.800000190734863,0.6,1.8,239.0,null,null,67.0,Torroja del Priorat,43682


In [0]:
display(pivotRenamedDF.filter(F.col("municipio") == "Amposta"))

fecha,indicativo,nombre,provincia,prec,tmax,tmed,tmin,velmedia,racha,sol,presMax,presMin,rhum,municipio,cod_municipio
2018-03-30T12:00:00.000,UU,Amposta,Tarragona,0.0,18.700000762939453,17.799999237060547,17.100000381469727,1.6,5.2,787.0,1004.2999877929688,1003.9000244140625,28.0,Amposta,43515
2018-01-10T12:00:00.000,UU,Amposta,Tarragona,0.0,15.800000190734863,15.199999809265137,14.199999809265137,0.9,2.9,464.0,1015.0999755859375,1014.7000122070312,43.0,Amposta,43515
2018-07-09T12:00:00.000,UU,Amposta,Tarragona,0.0,28.5,28.299999237060547,27.700000762939453,4.8,6.6,714.0,1016.7999877929688,1016.5,59.0,Amposta,43515
2017-09-23T12:00:00.000,UU,Amposta,Tarragona,0.0,24.799999237060547,24.0,23.299999237060547,2.9,4.9,422.0,1020.4000244140625,1020.0999755859375,64.0,Amposta,43515
2017-01-15T12:00:00.000,UU,Amposta,Tarragona,0.0,12.199999809265137,12.0,11.699999809265137,7.5,11.7,372.0,1021.4000244140625,1021.0,42.0,Amposta,43515
2016-06-26T12:00:00.000,UU,Amposta,Tarragona,0.0,25.299999237060547,24.899999618530273,24.700000762939453,4.5,6.7,954.0,1018.4000244140625,1018.0999755859375,61.0,Amposta,43515
2016-02-14T12:00:00.000,UU,Amposta,Tarragona,0.0,17.200000762939453,16.899999618530273,16.5,4.4,7.1,637.0,997.2000122070312,996.5,34.0,Amposta,43515
2016-06-29T12:00:00.000,UU,Amposta,Tarragona,0.0,25.600000381469727,25.299999237060547,25.100000381469727,3.3,5.2,838.0,1015.7000122070312,1015.5,70.0,Amposta,43515
2020-02-17T12:00:00.000,UU,Amposta,Tarragona,0.0,16.5,16.100000381469727,15.800000190734863,5.0,6.4,615.0,1024.0999755859375,1023.7000122070312,82.0,Amposta,43515
2019-10-05T12:00:00.000,UU,Amposta,Tarragona,0.0,26.100000381469727,25.899999618530273,25.600000381469727,2.1,5.0,701.0,1015.2999877929688,1014.9000244140625,50.0,Amposta,43515


# AEMET

In [0]:
valores_aemetDF = spark.read.parquet("/mnt/IncendiosForestalesCAT/raw/aemet/valores_clima_diarios")\
.filter(F.col("provincia").isin(["BARCELONA", "TARRAGONA", "GIRONA", "LLEIDA"]))

columns = ["fecha", "indicativo", "nombre","provincia", "prec", "tmax", "tmed", "tmin", "velmedia", "racha", "sol", "presMax", "presMin", "rhum", "fuente"]
valores_aemetDF = valores_aemetDF.withColumn("fuente", F.lit("aemet")).withColumn("rhum", F.lit(None)).select(columns)\
.withColumn('prec', F.regexp_replace('prec', ',', '.'))\
.withColumn('tmax', F.regexp_replace('tmax', ',', '.'))\
.withColumn('tmed', F.regexp_replace('tmed', ',', '.'))\
.withColumn('tmin', F.regexp_replace('tmin', ',', '.'))\
.withColumn('velmedia', F.regexp_replace('velmedia', ',', '.'))\
.withColumn('racha', F.regexp_replace('racha', ',', '.'))\
.withColumn('sol', F.regexp_replace('sol', ',', '.'))\
.withColumn('presMax', F.regexp_replace('presMax', ',', '.'))\
.withColumn('presMin', F.regexp_replace('presMin', ',', '.'))

display(valores_aemetDF)

fecha,indicativo,nombre,provincia,prec,tmax,tmed,tmin,velmedia,racha,sol,presMax,presMin,rhum,fuente
2016-01-01,9698U,TALARN,LLEIDA,1.4,8.2,5.7,3.2,1.1,3.3,0.3,931.1,923.7,null,aemet
2016-01-01,0002I,VANDELLÒS,TARRAGONA,0.1,16.4,13.6,10.8,0.6,3.6,null,null,null,null,aemet
2016-01-01,0016A,REUS AEROPUERTO,TARRAGONA,0.4,17.0,11.8,6.7,0.8,5.0,0.3,1016.4,1007.8,null,aemet
2016-01-01,0367,GIRONA AEROPUERTO,GIRONA,0.0,15.7,9.5,3.3,4.2,12.5,2.3,1009.5,998.8,null,aemet
2016-01-01,9720X,TÀRREGA,LLEIDA,0.0,null,null,null,0.8,3.9,null,null,null,null,aemet
2016-01-01,0324A,RIPOLL,GIRONA,0.0,11.4,6.8,2.3,1.1,6.9,null,null,null,null,aemet
2016-01-01,9619,LA SEU D'URGELL,LLEIDA,0.6,8.4,4.6,0.7,null,null,null,null,null,null,aemet
2016-01-01,0076,BARCELONA AEROPUERTO,BARCELONA,0.0,16.4,12.4,8.3,3.9,9.2,0.0,1025.3,1015.8,null,aemet
2016-01-01,9990X,"NAUT ARAN, ARTIES",LLEIDA,1.0,14.5,8.4,2.2,1.1,7.2,null,null,null,null,aemet
2016-01-01,9981A,ESTACIÓN DE TORTOSA (ROQUETES),TARRAGONA,0.0,17.2,13.4,9.6,1.9,7.2,2.2,1019.5,1011.0,null,aemet


In [0]:
estaciones_aemetDF = spark.read.parquet("/mnt/IncendiosForestalesCAT/raw/aemet/estaciones")\
.filter(F.col("provincia").isin(["BARCELONA", "TARRAGONA", "GIRONA", "LLEIDA"]))
display(estaciones_aemetDF)

altitud,indicativo,indsinop,latitud,longitud,nombre,provincia
74,0252D,08186,413515N,023224E,ARENYS DE MAR,BARCELONA
4,0076,08181,411734N,020412E,BARCELONA AEROPUERTO,BARCELONA
408,0200E,,412506N,020727E,"BARCELONA, FABRA",BARCELONA
6,0201D,08180,412326N,021200E,BARCELONA,BARCELONA
291,0149X,08174,414312N,015025E,MANRESA,BARCELONA
146,0229I,08192,413125N,020611E,SABADELL AEROPUERTO,BARCELONA
40,0255B,08188,413903N,024149E,SANTA SUSANNA,BARCELONA
143,0367,08184,415442N,024548E,GIRONA AEROPUERTO,GIRONA
95,0370B,,415848N,024931E,"GIRONA, ANTIC INSTITUT",GIRONA
157,0372C,08120,420616N,024549E,PORQUERES,GIRONA


## Añadir cod_municipio INE y municipio a estaciones aemet

In [0]:
import pandas as pd
estaciones_aemetPD = estaciones_aemetDF.toPandas()

estaciones_aemetPD['municipio']=["Arenys de Mar","Barcelona","Barcelona","Barcelona","Manresa","Sabadell","Santa Susanna","Grirona","Grirona","Grirona","Ripoll","Reus","Vandellos l’Hospitalet de l’Infant","Alp","Seu d'Urgell, La","Lleida","Lleida","Naut aran","Talarn","Tarrega","Roquetes","Sant Jaume d'Enveja"]
estaciones_aemetPD['cod_municipio']=[8006,8019,8019,8019,8113,8187,8261,17079,17079,17079,17147,43123,43162,17006,25203,25119,25119,25025,25215,25217,43133,43902]
estaciones_aemetDF= spark.createDataFrame(estaciones_aemetPD)

## JUNTAR VALORES CLIMA DIARIOS AEMET + ESTACIONES AEMET

In [0]:
valores_aemetDF = valores_aemetDF.alias("vdAemet").join(estaciones_aemetDF.alias("eAemet"), F.col("vdAemet.indicativo")== F.col("eAemet.indicativo"),"inner").select(["vdAemet.*", "eAemet.municipio","eAemet.cod_municipio"])

## JUNTAR VALORES METEO AEMET + METEOCAT

**Se unen los valores de AEMET y METEOCAT y se crea un campo "Fuente", en caso de querer conocer a que fuente corresponde**

In [0]:
columns = ["fecha", "indicativo", "nombre","provincia", "prec", "tmax", "tmed", "tmin", "velmedia", "racha", "sol", "presMax", "presMin", "rhum", "fuente", "municipio", "cod_municipio"]
valores_aemet_meteocatDF = valores_aemetDF.union(pivotRenamedDF.withColumn("fuente", F.lit("meteocat")).select(columns))
valores_aemet_meteocatDF = reduce(lambda dfl, x: dfl.withColumn(x[0], F.col(x[0]).cast(x[1])), pivotRenamedDF.dtypes, valores_aemet_meteocatDF)

In [0]:
estaciones_meteocatDF.count()

Out[12]: 233

In [0]:
pivotDF.count()

Out[13]: 480026

In [0]:
pivotRenamedDF.count()

Out[14]: 478469

In [0]:
valores_aemetDF.count()

Out[15]: 45677

In [0]:
valores_aemet_meteocatDF.count()

Out[16]: 524146

In [0]:
valores_aemet_meteocatDF.write.mode("overwrite").parquet(f"/mnt/IncendiosForestalesCAT/prep/aemet_meteocat/valores_clima_diarios/")

In [0]:
display(valores_aemet_meteocatDF)

fecha,indicativo,nombre,provincia,prec,tmax,tmed,tmin,velmedia,racha,sol,presMax,presMin,rhum,fuente,municipio,cod_municipio
2020-09-30,0252D,ARENYS DE MAR,BARCELONA,0.0,24.7,19.2,13.6,1.4,8.1,null,null,null,null,aemet,Arenys de Mar,8006
2020-09-29,0252D,ARENYS DE MAR,BARCELONA,0.0,24.7,19.4,14.0,1.1,4.4,null,null,null,null,aemet,Arenys de Mar,8006
2020-09-28,0252D,ARENYS DE MAR,BARCELONA,0.0,23.4,18.6,13.8,1.4,5.0,null,null,null,null,aemet,Arenys de Mar,8006
2020-09-27,0252D,ARENYS DE MAR,BARCELONA,0.0,21.4,15.4,9.5,0.8,11.1,null,null,null,null,aemet,Arenys de Mar,8006
2020-09-26,0252D,ARENYS DE MAR,BARCELONA,0.1,21.8,16.5,11.2,1.7,9.2,null,null,null,null,aemet,Arenys de Mar,8006
2020-09-25,0252D,ARENYS DE MAR,BARCELONA,0.0,24.5,19.2,14.0,2.5,12.5,null,null,null,null,aemet,Arenys de Mar,8006
2020-09-24,0252D,ARENYS DE MAR,BARCELONA,0.0,28.1,22.5,16.9,1.4,8.9,null,null,null,null,aemet,Arenys de Mar,8006
2020-09-23,0252D,ARENYS DE MAR,BARCELONA,0.1,26.9,22.0,17.1,1.7,8.9,null,null,null,null,aemet,Arenys de Mar,8006
2020-09-22,0252D,ARENYS DE MAR,BARCELONA,18.1,26.8,22.1,17.4,0.6,7.5,null,null,null,null,aemet,Arenys de Mar,8006
2020-09-21,0252D,ARENYS DE MAR,BARCELONA,2.3,24.5,21.1,17.7,0.3,3.3,null,null,null,null,aemet,Arenys de Mar,8006
